# GROMACS molecular dynamics demonstration

Molecular dynamics (MD) is a versatile method for simulating the time-dependent behavior of molecular systems. This notebook demonstrates how MD simulations can be run using the [GROMACS](https://www.gromacs.org/) software to study the self-assembly of lipids. To accelerate the simulations, a coarse-grained (CG) model using the [Martini force field](http://cgmartini.nl/) is employed. Here, multiple atoms are condensed into larger beads ("superatoms"), decreasing the computational cost of the simulations compared to simulating each atom explicitly (all-atom representation). In addition, parallel processing with multiple CPU cores is leveraged to demonstrate the principle of high-performance computing (HPC).

## Preparations

First, we need to make some choices regarding our MD simulation:

* What kind of lipid do we want to study and how many?
* What solvent do we want to use?
* How large (cubic) box do we want to simulate?

To provide these selections, run the configuration script below. For reference, W = water, EOL = ethanol and OD = octadecane. Lipid names are long and complicated, but feel free to search those up on the internet. The box size is the side length of the cubic cell in nanometers.

In [ ]:
%run config.py

The input parameters are stored in the variables `lipid.value`, `solvent.value`, `nlip.value` and `box.value`, respectively.

## Building the solvated lipid system

The solvated lipid box will be constructed in the following. First, let's try inserting the selected lipids into a cubic simulation box of specified size. This is done using the GROMACS tool `insert-molecules` and the resulting system will be stored in a file named `lipids.gro`.

Note: if you want to know more about the different commands and flags passed to GROMACS, there's a command-line reference available in the [official GROMACS documentation](https://manual.gromacs.org/current/user-guide/cmdline.html).

In [ ]:
!gmx_mpi insert-molecules -o lipids.gro -ci ff22/{lipid.value}.gro -nmol {nlip.value} -box {box.value} -try 200

GROMACS will dump quite a lot of output. The most important content is usually printed toward the bottom. Always make sure to check for any error messages. If you get an error, read the message and try to figure out what went wrong. A collection of common runtime errors is available in the [GROMACS documentation](https://manual.gromacs.org/current/user-guide/run-time-errors.html).

Next, we need to fill the `lipids.gro` system with the chosen solvent. As much solvent as will fit in the box containing the lipids is automatically added. This is done using the `solvate` tool of GROMACS and the resulting system will be stored in the file `solvated-lipids.gro`.

In [ ]:
!gmx_mpi solvate -cp lipids.gro -cs ff22/{solvent.value}.gro -o solvated-lipids.gro

Great, now we have a box filled with lipids and solvent of your choice.

## The topology

To be able to perform the actual MD simulation, we need a file describing on which atoms and atom combinations (pairs, triples, quadruples) the different components of the force field should act and how (which parameters should be applied in each case). All this is contained in the topology file.

A template topology file is provided. However, before it can be used we need to fix some absolute paths in the file to point to the applied coarse-grained force field. At the same time, we need to specify the selected lipid type and solvent, as well as how many molecules of each were respectively added. To this end, check the previous `insert-molecules` and `solvate` outputs for the number of lipids and solvent molecules that were *de facto* added. Paste these numbers after the `{lipid.value}` and `{solvent.value}` variables in the command below (replace `N_LIPID_MOLS` and `N_SOLVENT_MOLS`).

The editing of the topology file `topol.top` is done automatically by the `init_files.sh` script below (feel free to have a look what it does under the hood with `!cat init_files.sh`). The script will also initialize input files needed for following energy minimization and MD steps.

**Note! If the number of lipid/solvent molecules in the topology does not match the structure file, the next steps will fail. So make sure to edit the numbers below carefully.**

In [ ]:
!bash init_files.sh {lipid.value} N_LIPID_MOLS {solvent.value} N_SOLVENT_MOLS

## Energy minimization

Before running the actual molecular dynamics, we need to ensure that the structure of our system is reasonable (no clashing atoms or otherwise inappropriate geometry). This is done by performing an *energy minimization* run where the molecular structure is relaxed by minimizing the forces acting on the atoms.

Before this, we need to assemble the simulation parameters (`em.mdp`), topology (`topol.top`) and structure (`solvated-lipids.gro`) of our system into a binary input file `em.tpr`. This is accomplished using the `grompp` pre-processing tool of GROMACS.

In [ ]:
!gmx_mpi grompp -f em.mdp -c solvated-lipids.gro -p topol.top -o em.tpr -maxwarn 10

Now that we have our binary input file, let's start the energy minimzation using `mdrun`.

In [ ]:
!gmx_mpi mdrun -s em.tpr -v

Let's visualize the variation of the potential energy during the energy minimization. Run the command below to extract the data from the `ener.edr` output and store it into a file named `energy.xvg`.

In [ ]:
!gmx_mpi energy -f ener.edr -o energy.xvg <<< "5"

Plot the data using `matplotlib` as shown below.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

energy = np.loadtxt('energy.xvg', comments=['#', '@'])
plt.plot(energy[50:, 0], energy[50:, 1])
plt.xlabel('Energy minimization step')
plt.ylabel('Potential energy (kJ/mol)')
plt.show()

We can see that the potential energy of our system decreases nicely over the performed 1000 energy minimization steps. Now that our starting configuration is a bit more stable, we're ready to start preparing the inputs for the MD simulation.

## Molecular dynamics

Before starting the MD simulation, we need to again pre-process our energy minimized structure (`confout.gro`), topology (`topol.top`) and MD simulation parameters (`md.mdp`) into a binary input file `md.tpr` using the `grompp` command below.

In [ ]:
!gmx_mpi grompp -f md.mdp -c confout.gro -p topol.top -o md.tpr -maxwarn 10

The MD simulation will be accelerated using multiple CPU cores. `orterun -n $SLURM_CPUS_PER_TASK` will be used to tell the job to use all cores that were requested when this notebook was started.

The run will take a few tens of seconds to a couple of minutes. Feel free to change `$SLURM_CPUS_PER_TASK` to a smaller number. How does it affect the simulation wall-time and performance (ns/day)? These are printed at the end of the output.

Running simulations in parallel using several CPU cores that are each responsible for a subset of the total computational task is the fundamental principle of high-performance computing!

In [ ]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -deffnm md -v

## Visualization

Nice! You have performed a molecular dynamics simulation of a box of solvated lipids. Now, the most interesting part is to see what happens during the simulation. However, before visualizing the trajectory stored in `md.xtc` we need to fix some connectivity issues. The simulation we ran exploited the concept of *periodic boundary conditions* (PBC), meaning that molecules can cross the system boundaries and end up on the opposite side of the cell – a convenient trick for modeling extended condensed phases. Depending on how you visualize your system (bonds included or not), a side-effect may be that some bonds seem to artificially stretch across the simulation cell. The following command will fix this potential issue.

In [ ]:
!gmx_mpi trjconv -f md.xtc -s md.tpr -o md_whole.xtc -pbc mol <<< "0"

Let's now visualize our trajectory using the `MDAnalysis` and `nglview` tools. Gray spheres will denote coarse-grained beads belonging to the lipid tails, whereas white and colored spheres denote the head groups.

In [ ]:
import MDAnalysis as mda
import nglview as nv

u = mda.Universe('md.gro', 'md_whole.xtc')
view = nv.show_mdanalysis(u)
view.add_spacefill(lipid.value)
view.add_point(solvent.value)
view.add_unitcell()
view

## Thermodynamic and geometric interpretation

What happened? In the first frame of the simulation the lipids are more or less randomly scattered within the box. As you scroll to the end, they seem to be organizing somehow (note the periodic boundary conditions!). To understand the thermodynamics behind this process, one needs consider the *amphiphilic* character of lipids – they have a hydrophilic head group and a hydrophobic (lipophilic) tail. Depending on the solvent, they self-assemble in a way that minimizes their *free energy* $\Delta G$. In essence, polar head groups tend to cluster together if the solvent is non-polar (like octadecane) whereas the hydrophobic tails favorably embed within the solvent. On the other hand, if the solvent is polar (like water), the lipophilic tails group together while the head groups orient toward the solvent.

The process of self-assembly is *spontaneous* because the system can decrease its free energy by aggregating in a specific way. Typically systems tend toward a state of maximal entropy, but in this case the system is becoming more ordered so the entropy is strangely decreasing! The free energy is, however, crucially affected also by the enthalpy change, which in this case is highly negative as the aggregation results in a more optimal electrostatic interaction. This yields an overall negative free energy change, 

$$\Delta G=\Delta H-T\Delta S<0$$

Besides the solvent-dictated orientation of head groups and tails, the geometry of the lipid affects what kind of structures form. The critial packing parameter (CPP) is typically used to predict the morphology qualitatively,

$$\mathrm{CPP}=\frac{V_t}{A_h\cdot l_t}$$

where $V_t$ and $l_t$ are the volume and length of the tail, respectively, whereas $A_h$ denotes the cross-sectional surface area of the head group. Assuming a polar solvent, lipids with large head groups ($\mathrm{CPP}\leq1/3$) typically form spherical micelles, whereas cylinders and vesicles are possible with slightly smaller heads ($1/3<\mathrm{CPP}\leq1/2$, e.g. LPC). If the lipid is rather cylindrical ($1/2<\mathrm{CPP}\leq1$, e.g. DPPC), various bilayers are likely to form. Conversely, in non-polar solvents lipids self-assemble into inverted micelles. Inversion is also possible in a polar solvent if the head group is very small ($\mathrm{CPP}>1$, e.g. POPE), yielding self-assembled structures where the solvent is essentially encapsulated within cavities bordered by hydrophilic heads.

## Rinse and repeat

Try changing the simulated lipid and/or solvent and see if you can reproduce some of the other self-assembled structures mentioned above. Note that the concentration also affects the results – formation of large structures such as bilayers is unlikely if the concentration of lipids is too low.

To start from a fresh table, run the script below to wipe all files that were created up to this point.

In [ ]:
!bash clean.sh